In [ ]:
"""
Purpose: Figure out the following: 
1) Had to add a second point to the output
2) Why in high crossovers some nodes that will be errored out are put in blue

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-06-02 14:13:41,802 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-02 14:13:41,814 - settings - Setting database.user to celiib
INFO - 2021-06-02 14:13:41,815 - settings - Setting database.password to newceliipass
INFO - 2021-06-02 14:13:41,823 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-06-02 14:13:41,824 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-02 14:13:41,835 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-06-02 14:13:42,263 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-02 14:13:42,268 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-02 14:13:42,271 - settings - Setting database.user to celiib
INFO - 2021-06-02 14:13:42,274 - settings - Setting database.password to newceliipass
INFO - 2021-06-02 14:13:42,276 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-02 14:13:42,277 - settings - Setting database.user to celiib
INFO - 2021-06-02 14:13:42,280 - settings - Setting database.password to newceliipass
INFO - 2021-06-02 14:13:42,282 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-02 14:13:42,760 - settings - Setting enable_python_native_blobs to True


In [3]:
import neuron_visualizations as nviz
import neuron_utils as nru
import proofreading_utils as pru
import time
import system_utils as su
import axon_utils as au
import trimesh_utils as tu
import allen_utils as alu

In [4]:
minnie,_ = du.configure_minnie_vm()

INFO - 2021-06-02 14:13:47,160 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-06-02 14:13:47,162 - settings - Setting database.user to celiib
INFO - 2021-06-02 14:13:47,163 - settings - Setting database.password to newceliipass
INFO - 2021-06-02 14:13:47,165 - settings - Setting enable_python_native_blobs to True
INFO - 2021-06-02 14:13:47,740 - settings - Setting enable_python_native_blobs to True


# Getting the Neuron

In [5]:
segment_id = 864691134884759034

In [6]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,split_index=0)

**Using table __decomposition_axon for table_to_neuron_objs**
Decompressing Neuron in minimal output mode...please wait
Dont need spine recalculation
Number of Neurons found = 1


In [7]:
nviz.visualize_neuron(neuron_obj,limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [ ]:
pru.exc_high_degree_branching_filter["filter_function"]

In [8]:
import error_detection as ed
output_info = pru.filter_away_limb_branch_dict_with_function(neuron_obj,
                 limb_branch_dict_function=ed.high_degree_branch_errors_limb_branch_dict,
                return_error_info=True,
                plot_limb_branch_filter_away=True,
                plot_limb_branch_filter_with_disconnect_effect=True,
                 plot_final_neuron=False,
                return_limb_branch_dict_to_cancel=True,)



 ----- Working on coordinate 0: [1142791.55987193  642687.69846821 1053519.58719435]--------
Changing max_degree_to_resolve = 8 because upstream width was 206.069558989696 
winning_downstream = 16,error_downstream = [12 17] 
coordinate [1142791.55987193  642687.69846821 1053519.58719435] had error branches [12 17]--------


 ----- Working on coordinate 1: [1143307.14466577  647958.48340038 1054819.23265028]--------
winning_downstream = 18,error_downstream = [10 19] 
coordinate [1143307.14466577  647958.48340038 1054819.23265028] had error branches [10 19]--------


 ----- Working on coordinate 2: [1167531.54936081  749545.32987519 1092171.4209236 ]--------
winning_downstream = 55,error_downstream = [33 56] 
coordinate [1167531.54936081  749545.32987519 1092171.4209236 ] had error branches [33 56]--------


 ----- Working on coordinate 3: [1201118.94568997  774313.12277884 1076556.144653  ]--------
winning_downstream = 74,error_downstream = [76 77 79] 
coordinate [1201118.94568997  77



Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh


In [10]:
limb_branch_dict = output_info[-1]
limb_branch_dict

{'L1': array([ 12,  17,  10,  19,  33,  56,  76,  77,  79,  84,  89,  95,  98,
         99,  93, 105])}

In [14]:
nviz.plot_limb_branch_dict(neuron_obj,limb_branch_dict)


 Working on visualization type: mesh


# Mockig up the Crossover

In [ ]:
xu.edges_and_weights_to_graph

In [78]:
"""
Purpose: To pair branches of a subgraph
together if they match skeleton angles 
or some other criteria

Application: for grouping 
red/blue splits together

Arguments: 
1) Limb object
2) branches to check for connectivity (or the connected components precomputed)

Pseudocode: 
0) Compute the connected components if not already done
1) For each connected component: 
a. Find path of connected component back to the starting node
b. If path only of size 1 then just return either error branches or connected components
c. Get the border error branch and the border parent branch from the path
d) add the border error branch to a dictionary mapping parent to border branch and the conn comp it belongs to

2) For each parent branch:
- if list is longer than 1
  a. match the border error branches to each other to see if should be connected
  ( have argument to set the function to use for this)
  b. If have any matches then add the pairings to a list of lists, else add to a seperate list

3. Use the pairings to create new connected components if any should be combined

"""
import networkx as nx
import matplotlib.pyplot as plt
import networkx_utils as xu
import numpy_utils as nu

limb_obj = neuron_obj[1]
branches = limb_branch_dict["L1"]
conn_comp = None
plot_conn_comp_before_combining = False
pair_method = "skeleton_angle"

#for the skeleton angle pairing
match_threshold = 70
thick_width_threshold = 200
comparison_distance_thick = 3000
offset_thick = 1000
comparison_distance_thin = 1500
offset_thin = 0

verbose = True
plot_conn_comp_before_combining = False


G = nx.Graph(limb_obj.concept_network_directional)
#0) Compute the connected components if not already done
if conn_comp is None:
    conn_comp = xu.connected_components(G.subgraph(branches))
else:
    if len(conn_comp)>0:
        branches = nu.concatenate_lists(conn_comp)
    else:
        branches = []

if verbose:
    print(f"conn_comp = {conn_comp}")
    print(f"branches = {branches}")
    
if plot_conn_comp_before_combining:
    print(f"plot connected components before combined")
    nx.draw(G.subgraph(branches),with_labels=True)
    plt.show()
    
"""    
1) For each connected component: 
a. Find path of connected component back to the starting node
b. If path only of size 1 then just return either error branches or connected components
c. Get the border error branch and the border parent branch from the path
d) add the border error branch to a dictionary mapping parent to border branch and the conn comp it belongs to
"""
parent_to_branch_borders = dict()

for j,c in enumerate(conn_comp):
    short_path,st,end = xu.shortest_path_between_two_sets_of_nodes(G,c,[limb_obj.current_starting_node])
#     if verbose:
#         print(f"short_path,st,end= {short_path,st,end}")
        
    if len(short_path)<2:
        continue
    
    parent_border = short_path[1]
    branch_border = st
    
    if parent_border not in parent_to_branch_borders:
        parent_to_branch_borders[parent_border] = dict(branch_idx = [branch_border],
                                                     conn_comp_idx = [j],
                                                      branch_to_comp_map = {branch_border:j})
    else:
        parent_to_branch_borders[parent_border]["branch_idx"].append(branch_border)
        parent_to_branch_borders[parent_border]["conn_comp_idx"].append(j)
        parent_to_branch_borders[parent_border]["branch_to_comp_map"][branch_border] = j
        
        
if verbose:
    print(f"parent_to_branch_borders = {parent_to_branch_borders}")

"""
2) For each parent branch:
- if list is longer than 1
  a. match the border error branches to each other to see if should be connected
  ( have argument to set the function to use for this)
  b. If have any matches then add the pairings to a list of lists, else add to a seperate list
"""
conn_comp_pairings = []
for parent_idx,border_info in parent_to_branch_borders.items():
    if len(border_info["branch_idx"])<2:
        continue
    
    #a. match the border error branches to each other to see if should be connected
    if pair_method == "pair_all":
        conn_comp_pairings.append(border_info["conn_comp_idx"])
        #conn_comp_pairings.append(border_info["branch_idx"])
    elif pair_method == "skeleton_angle":
        parent_width = nru.width(limb_obj[parent_idx])
        
        if parent_width > thick_width_threshold:
            comparison_distance = comparison_distance_thick
            offset = offset_thick
        else:
            comparison_distance = comparison_distance_thin
            offset = offset_thin
            
        
        curr_branches = border_info["branch_idx"]
        try:
            matched_edges, matched_edges_angles = ed.matched_branches_by_angle(limb_obj,
                                                           branches = border_info["branch_idx"],
                                                        offset=offset,
                                                        comparison_distance = comparison_distance,
                                                        match_threshold = match_threshold,
                                                        verbose = False,
                                                        plot_intermediates = False,
                                                        plot_match_intermediates = False,
                                                        less_than_threshold = True
                                                        )
        except Exception as e:
            if verbose:
                print(f"Hit an error when doing matched_branches_by_angle: {str(e)}")
            matched_edges = []
        
        if len(matched_edges)>0:
            match_G = xu.edges_and_weights_to_graph(matched_edges)
            match_G_conn_comp = xu.connected_components(match_G)
            for m_comp in match_G_conn_comp:
                m_comp_conn_idx = [border_info["branch_to_comp_map"][k] for k in m_comp]
            
                conn_comp_pairings.append(m_comp_conn_idx)
        
        
        if verbose:
            print(f"parent_width = {parent_width}, comparison_distance = {comparison_distance}, offset = {offset}")
            print(f"matched_edges = {matched_edges}")
    else:
        raise Exception(f"Unimplemented pair_method = {pair_method}")
        
if verbose:
    print(f"conn_comp_pairings = {conn_comp_pairings}")
    
"""
3. Use the pairings to create new connected components if any should be combined

"""
all_conn_comp_idx = np.arange(len(conn_comp))
conn_comp_ar = np.array(conn_comp)
final_comp_comp_from_pair = [list(np.concatenate(conn_comp_ar[k]))
                             for k in conn_comp_pairings]
if len(final_comp_comp_from_pair)>0:
    conn_comp_idx_from_pairs = np.hstack(conn_comp_pairings)
else:
    conn_comp_idx_from_pairs = []
    
conn_comp_idx_leftover = np.delete(all_conn_comp_idx,conn_comp_idx_from_pairs)
conn_comp_from_leftover = [conn_comp[k] for k in conn_comp_idx_leftover]

conn_comp_combined = final_comp_comp_from_pair + conn_comp_from_leftover


if verbose:
    print(f"final_comp_comp_from_pair = {final_comp_comp_from_pair}")
    print(f"conn_comp_idx_leftover = {conn_comp_idx_leftover}")
    print(f"conn_comp_from_leftover = {conn_comp_from_leftover}")
    print(f"conn_comp_combined = {conn_comp_combined}")


    

conn_comp = [[33], [98], [99], [105], [10], [12], [76], [77], [79], [17], [19], [84], [56], [89], [93], [95]]
branches = [ 12  17  10  19  33  56  76  77  79  84  89  95  98  99  93 105]
parent_to_branch_borders = {57: {'branch_idx': [33, 56], 'conn_comp_idx': [0, 12], 'branch_to_comp_map': {33: 0, 56: 12}}, 88: {'branch_idx': [98, 99, 95], 'conn_comp_idx': [1, 2, 15], 'branch_to_comp_map': {98: 1, 99: 2, 95: 15}}, 107: {'branch_idx': [105, 93], 'conn_comp_idx': [3, 14], 'branch_to_comp_map': {105: 3, 93: 14}}, 16: {'branch_idx': [10, 19], 'conn_comp_idx': [4, 10], 'branch_to_comp_map': {10: 4, 19: 10}}, 5: {'branch_idx': [12, 17], 'conn_comp_idx': [5, 9], 'branch_to_comp_map': {12: 5, 17: 9}}, 78: {'branch_idx': [76, 77, 79], 'conn_comp_idx': [6, 7, 8], 'branch_to_comp_map': {76: 6, 77: 7, 79: 8}}, 90: {'branch_idx': [84, 89], 'conn_comp_idx': [11, 13], 'branch_to_comp_map': {84: 11, 89: 13}}}
parent_width = 61.74004650041857, comparison_distance = 1500, offset = 0
matched_edges = [[3

In [87]:
G = limb_obj.concept_network_directional

[[33],
 [98],
 [99],
 [105],
 [10],
 [12],
 [76],
 [77],
 [79],
 [17],
 [19],
 [84],
 [56],
 [89],
 [93],
 [95]]

In [90]:
nru.pair_branch_connected_components(limb_obj=neuron_obj[1],
    #branches = limb_branch_dict["L1"],
    conn_comp = xu.connected_components(limb_obj.concept_network_directional.subgraph(limb_branch_dict["L1"]),
                       ),
    plot_conn_comp_before_combining = False,
                                verbose = True)

conn_comp = [[33], [98], [99], [105], [10], [12], [76], [77], [79], [17], [19], [84], [56], [89], [93], [95]]
branches = [ 33  98  99 105  10  12  76  77  79  17  19  84  56  89  93  95]
parent_to_branch_borders = {57: {'branch_idx': [33, 56], 'conn_comp_idx': [0, 12], 'branch_to_comp_map': {33: 0, 56: 12}}, 88: {'branch_idx': [98, 99, 95], 'conn_comp_idx': [1, 2, 15], 'branch_to_comp_map': {98: 1, 99: 2, 95: 15}}, 107: {'branch_idx': [105, 93], 'conn_comp_idx': [3, 14], 'branch_to_comp_map': {105: 3, 93: 14}}, 16: {'branch_idx': [10, 19], 'conn_comp_idx': [4, 10], 'branch_to_comp_map': {10: 4, 19: 10}}, 5: {'branch_idx': [12, 17], 'conn_comp_idx': [5, 9], 'branch_to_comp_map': {12: 5, 17: 9}}, 78: {'branch_idx': [76, 77, 79], 'conn_comp_idx': [6, 7, 8], 'branch_to_comp_map': {76: 6, 77: 7, 79: 8}}, 90: {'branch_idx': [84, 89], 'conn_comp_idx': [11, 13], 'branch_to_comp_map': {84: 11, 89: 13}}}
parent_width = 61.74004650041857, comparison_distance = 1500, offset = 0
matched_edges = [[3

[[56, 33],
 [98, 99, 95],
 [105, 93],
 [76, 77, 79],
 [10],
 [12],
 [17],
 [19],
 [84],
 [89]]

In [91]:
nviz.plot_limb_branch_dict(neuron_obj,
                          #dict(L1=[10,12,17,19,84,89]),
                          dict(L1=[25,33,98,99,95,105,93,76,77,79]))


 Working on visualization type: mesh


# Doing the Red blue suggestions

In [ ]:
#apply_proofreading_filters_to_neuron is where do the disconnect

In [92]:
limb_branch_dict_to_cancel = output_info[-1]
limb_branch_dict_to_cancel

{'L1': array([ 12,  17,  10,  19,  33,  56,  76,  77,  79,  84,  89,  95,  98,
         99,  93, 105])}

In [100]:
red_blue_suggestions = pru.limb_branch_dict_to_cancel_to_red_blue_groups(neuron_obj=neuron_obj,
                                                limb_branch_dict_to_cancel=limb_branch_dict_to_cancel,
                                                            plot_all_blue_red_groups = False,
                                                            plot_final_blue_red_points = False,
                                                            red_blue_points_method = "closest_mesh_face",
                                                             verbose = True)

-- Working on L1 --
Error Branches = [ 12  17  10  19  33  56  76  77  79  84  89  95  98  99  93 105]

**conn_comp_errors = [[33], [98], [99], [105], [10], [12], [76], [77], [79], [17], [19], [84], [56], [89], [93], [95]]
Attempting to pair_conn_comp_errors
conn_comp = [[33], [98], [99], [105], [10], [12], [76], [77], [79], [17], [19], [84], [56], [89], [93], [95]]
branches = [ 33  98  99 105  10  12  76  77  79  17  19  84  56  89  93  95]
parent_to_branch_borders = {57: {'branch_idx': [33, 56], 'conn_comp_idx': [0, 12], 'branch_to_comp_map': {33: 0, 56: 12}}, 88: {'branch_idx': [98, 99, 95], 'conn_comp_idx': [1, 2, 15], 'branch_to_comp_map': {98: 1, 99: 2, 95: 15}}, 107: {'branch_idx': [105, 93], 'conn_comp_idx': [3, 14], 'branch_to_comp_map': {105: 3, 93: 14}}, 16: {'branch_idx': [10, 19], 'conn_comp_idx': [4, 10], 'branch_to_comp_map': {10: 4, 19: 10}}, 5: {'branch_idx': [12, 17], 'conn_comp_idx': [5, 9], 'branch_to_comp_map': {12: 5, 17: 9}}, 78: {'branch_idx': [76, 77, 79], 'con

blue L1 100 node offset skeleton coordinate is [1226556.59289792  752728.08208675 1054367.94380366]
 --> using closest_mesh_face method curr_points = [[1226529.33333333  752704.73333333 1054320.66666667]]
blue L1 100 node offset skeleton coordinate is [1226907.89692176  752686.21998175 1054051.07627292]
 --> using closest_mesh_face method curr_points = [[1226878.33333333  752665.3        1054053.        ]]
blue L1 100 node offset skeleton coordinate is [1227258.36987468  752417.18361594 1053997.33567742]
 --> using closest_mesh_face method curr_points = [[1227225.66666667  752356.3        1054057.        ]]

For Pair 4: coordinate [[1226371.   750978.9 1054649. ]]
red L1 97 node offset skeleton coordinate is [1226304.72273654  750528.51463992 1054731.44547309]
 --> using closest_mesh_face method curr_points = [[1226333.33333333  750530.66666667 1054705.        ]]
red L1 97 node offset skeleton coordinate is [1226126.96995518  749982.20601158 1054661.7935284 ]
 --> using closest_mesh_fa

Branch 79 had a valid upstream node 78 with common endpoint [[1201118.94568997  774313.12277884 1076556.144653  ]]
valid_upstream_branches = [78, 78, 78]
error_border_branches = [76, 77, 79]
error_border_coordinates = [array([[1201118.94568997,  774313.12277884, 1076556.144653  ]]), array([[1201118.94568997,  774313.12277884, 1076556.144653  ]]), array([[1201118.94568997,  774313.12277884, 1076556.144653  ]])]

Valid Branch 78 had a valid downstream node 74 with common endpoint [[1201118.94568997  774313.12277884 1076556.144653  ]]

After Adding Downstream Valid Nodes
error_border_branches = [76, 77, 79, -1]
valid_upstream_branches = [78, 78, 78, 74]
error_border_coordinates = [array([[1201118.94568997,  774313.12277884, 1076556.144653  ]]), array([[1201118.94568997,  774313.12277884, 1076556.144653  ]]), array([[1201118.94568997,  774313.12277884, 1076556.144653  ]]), array([[1201118.94568997,  774313.12277884, 1076556.144653  ]])]

Error Branch 77 had an error downstream node 80 with

blue L1 18 node offset skeleton coordinate is [1143514.05473084  648413.87579982 1054765.69923254]
 --> using closest_mesh_face method curr_points = [[1143583.33333333  648456.66666667 1054756.66666667]]
blue L1 18 node offset skeleton coordinate is [1143683.77079003  648856.38040101 1054639.48451319]
 --> using closest_mesh_face method curr_points = [[1143677.33333333  648762.96666667 1054657.        ]]
blue L1 18 node offset skeleton coordinate is [1143845.38040978  649371.98200352 1054525.30523928]
 --> using closest_mesh_face method curr_points = [[1143908.33333333  649356.5        1054509.33333333]]

For Pair 2: coordinate [[1139615.   648271.8 1060584. ]]
red L1 0 node offset skeleton coordinate is [1139229.36951556  648308.28772046 1061000.47587993]
 --> using closest_mesh_face method curr_points = [[1139211.66666667  648321.03333333 1060962.33333333]]
red L1 0 node offset skeleton coordinate is [1139109.30249891  648439.4519495  1061444.19164127]
 --> using closest_mesh_face me

blue L1 18 node offset skeleton coordinate is [1143845.38040978  649371.98200352 1054525.30523928]
 --> using closest_mesh_face method curr_points = [[1143908.33333333  649356.5        1054509.33333333]]

Working on connected component: [84]
Branch 84 had a valid upstream node 90 with common endpoint [[1214258.0878685   719470.78608745 1043268.48229103]]
valid_upstream_branches = [90]
error_border_branches = [84]
error_border_coordinates = [array([[1214258.0878685 ,  719470.78608745, 1043268.48229103]])]

Valid Branch 90 had a valid downstream node 86 with common endpoint [[1214258.0878685   719470.78608745 1043268.48229103]]

After Adding Downstream Valid Nodes
error_border_branches = [84, -1]
valid_upstream_branches = [90, 86]
error_border_coordinates = [array([[1214258.0878685 ,  719470.78608745, 1043268.48229103]]), array([[1214258.0878685 ,  719470.78608745, 1043268.48229103]])]


After Adding Downstream Error Nodes
error_border_branches = [84, -1]
valid_upstream_branches = [90, 8

In [102]:
red_blue_suggestions

{'L1': {0: {'error_branches': array([56, 33]),
   'error_branches_skeleton_length': 25083.529195765055,
   'valid_branches': array([57]),
   'parent_branch': 57,
   'parent_branch_width': 61.74004650041857,
   'parent_branch_axon': True,
   'n_error_branches': 2,
   'valid_points': array([[1167910.        ,  749521.33333333, 1091800.33333333],
          [1168409.        ,  749505.6       , 1091634.        ],
          [1168811.        ,  749297.56666667, 1091598.33333333],
          [1167248.33333333,  749722.93333333, 1092348.66666667],
          [1166886.33333333,  749972.66666667, 1091996.        ],
          [1166642.        ,  750377.46666667, 1092089.33333333]]),
   'error_points': array([[1167836.66666667,  749199.46666667, 1091781.        ],
          [1168201.        ,  749000.5       , 1091711.66666667],
          [1168489.        ,  748650.        , 1091527.33333333],
          [1167166.66666667,  749628.        , 1092584.        ],
          [1166697.66666667,  749760.23333